### Refining the objective
##### This is a script file which will target MoT Comp.
""" 
DataSet and files: 
- 5d067e6d8518921094fdab22_Accidents.csv

Problem to address: a solution to reduce # of accidents in Saudi Arabia
3 main areas:
1- Environment affects causing accidents
2- Drivers' behaviour causing accidents
3- Vehicle status causing accidents

Meta data has been provided to explain the data.
 
Developers&Auditers:

1-zs2002

2-aistif

3-mussa-mubarki

Date: Jun/18/2019"""

### Collect the data (save the data in the directory data/)

In [132]:
import numpy as np
import pandas as pd
# DF TO EXCEL
from pandas import ExcelWriter
import scipy.stats as stats
import requests
%config InlineBackend.figure_format = 'retina'
from datetime import datetime
from datetime import date
from datetime import time
from dateutil.parser import parse
from bs4 import BeautifulSoup
import html5lib

##### So here we have all data for 2017-2019 under //data folder

In [133]:
# reading the csv file whihc has been collected from https://services.amana-md.gov.sa/eservicesite/Inq/DeathInquiry.aspx
Accidents = pd.read_csv("data//5d067e6d8518921094fdab22_Accidents.csv", low_memory=False)

In [134]:
Accidents.shape

(36016, 28)

In [135]:
Accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36016 entries, 0 to 36015
Data columns (total 28 columns):
Unnamed: 0            36016 non-null int64
Accident_Time         36016 non-null object
Accident_Date_G       36016 non-null object
Rno                   36016 non-null int64
Road_Type             33798 non-null object
Region                36016 non-null object
Road_Status           21838 non-null object
Weather_Status        35569 non-null object
Dead_Number           24727 non-null float64
Injuries_Number       29384 non-null float64
Damage_Road_Type      5994 non-null object
Accident_Type         35740 non-null object
Car                   24219 non-null float64
Small_Truck           4958 non-null float64
Big_Truck             7949 non-null float64
Other_Car             333 non-null float64
Driver                27286 non-null float64
Way                   215 non-null float64
Vehicle               5836 non-null float64
Animal                418 non-null float64
Vehicles_Numb

In [136]:
# what have to change 'Unnamed: 0' to another column name
Accidents = Accidents.rename(columns={'Unnamed: 0': 'SequenceNo'})

In [137]:
# showing columns
Accidents_columns = Accidents.columns
Accidents.head(5)

,SequenceNo,Accident_Time,Accident_Date_G,Rno,Road_Type,Region,Road_Status,Weather_Status,Dead_Number,Injuries_Number,...,Vehicle,Animal,Vehicles_Number,Paints,Eyes,Road_Close_Days,Road_Close_Hours,GEOMETRIC_RoadTYPE,long,lat
0,0,5:00 AM,16/04/2019,425,سريع مع خدمة,Qassim,Good,good,0.0,1.0,...,NaN,NaN,1.0,1.0,1.0,NaN,NaN,Straight link,43.999093,26.426746
1,1,12:15 PM,16/04/2019,414,مزدوج,Qassim,Good,good,0.0,1.0,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,43.786877,26.057596
2,2,8:30 AM,15/04/2019,419,مزدوج,Qassim,Good,good,0.0,0.0,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,43.772701,26.345146
3,3,8:30 PM,15/04/2019,60,سريع مع خدمة,Qassim,Good,good,0.0,0.0,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,43.937996,26.205702
4,4,12:10 AM,14/04/2019,393,NaN,Hail,Good,good,NaN,NaN,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,41.702023,27.555157


### Data Cleaning and Data Exploration

#### Quality measures and dimensions
##### 1- completeness - filling missing data below
##### 2- consistency - can't be tested here
##### 3- conformity - can't be tested here
##### 4- Accuracy - assumed it's accurate since collected from MoT
##### 5- integrity - can't be tested here
##### 6- timeliness - can't be tested here

#### Data quality assurance

In [138]:
# verification for missing values for all columns
# Count the missing values — sum() works because True is 1 and False is 0.
Accidents_dic = {}
# show all columns with missing count
for one_col in Accidents.columns:
    # assign value to the key value calculate the total missing values then it to dict
    Accidents_dic[one_col] = "missing " + str(Accidents[one_col].isnull().sum())
print(Accidents_dic)
#Accidents_dic.keys()

{'Region': 'missing 0', 'lat': 'missing 8387', 'Road_Status': 'missing 14178', 'Other_Car': 'missing 35683', 'Car': 'missing 11797', 'Animal': 'missing 35598', 'Road_Close_Days': 'missing 35920', 'SequenceNo': 'missing 0', 'Vehicles_Number': 'missing 2356', 'Damage_Road_Type': 'missing 30022', 'Small_Truck': 'missing 31058', 'Big_Truck': 'missing 28067', 'Eyes': 'missing 254', 'Accident_Type': 'missing 276', 'GEOMETRIC_RoadTYPE': 'missing 17213', 'Accident_Time': 'missing 0', 'Weather_Status': 'missing 447', 'Paints': 'missing 247', 'Accident_Date_G': 'missing 0', 'Dead_Number': 'missing 11289', 'Driver': 'missing 8730', 'Road_Close_Hours': 'missing 35788', 'Way': 'missing 35801', 'Injuries_Number': 'missing 6632', 'long': 'missing 8387', 'Vehicle': 'missing 30180', 'Rno': 'missing 0', 'Road_Type': 'missing 2218'}


In [139]:
# This segment is for Quality checking of missing values only 
# casualities and stats
Accidents.Injuries_Number = Accidents.Injuries_Number.fillna(0)
Accidents.Dead_Number = Accidents.Dead_Number.fillna(0)
Accidents.Vehicles_Number = Accidents.Vehicles_Number.fillna(0)
# flags
Accidents.Eyes = Accidents.Eyes.fillna(0)
Accidents.Driver = Accidents.Driver.fillna(0)
Accidents.Vehicle = Accidents.Vehicle.fillna(0)
Accidents.Car = Accidents.Car.fillna(0)
Accidents.Other_Car = Accidents.Other_Car.fillna(0)
Accidents.Small_Truck = Accidents.Small_Truck.fillna(0)
Accidents.Big_Truck = Accidents.Big_Truck.fillna(0)
Accidents.Way = Accidents.Way.fillna(0)
Accidents.Paints = Accidents.Paints.fillna(0)
Accidents.Animal = Accidents.Animal.fillna(0)
# types and info
Accidents.Accident_Type = Accidents.Accident_Type.fillna('Unknown')
Accidents.Road_Status = Accidents.Road_Status.fillna('Unknown')
Accidents.Road_Close_Hours = Accidents.Road_Close_Hours.fillna(0)
Accidents.Road_Close_Days = Accidents.Road_Close_Days.fillna(0)
Accidents.Road_Type = Accidents.Road_Type.fillna('Unknown')
Accidents.Damage_Road_Type = Accidents.Damage_Road_Type.fillna('Unknown')
Accidents.GEOMETRIC_RoadTYPE = Accidents.GEOMETRIC_RoadTYPE.fillna('Unknown')
Accidents.long = Accidents.long.fillna(0)
Accidents.lat = Accidents.lat.fillna(0)
Accidents.Weather_Status = Accidents.Weather_Status.fillna('Unknown')

In [140]:
# This segment is for Quality checking of correct type

Accidents['Injuries_Number'] = (Accidents['Injuries_Number'].copy()).astype(int)
Accidents['Dead_Number'] = (Accidents['Dead_Number'].copy()).astype(int)
Accidents['Vehicles_Number'] = (Accidents['Vehicles_Number'].copy()).astype(int)
# keep them int altough they are flags
Accidents['Eyes'] = (Accidents['Eyes'].copy()).astype(int)
Accidents['Driver'] = (Accidents['Driver'].copy()).astype(int)
Accidents['Vehicle'] = (Accidents['Vehicle'].copy()).astype(int)
Accidents['Car'] = (Accidents['Car'].copy()).astype(int)
Accidents['Other_Car'] = (Accidents['Other_Car'].copy()).astype(int)
Accidents['Small_Truck'] = (Accidents['Small_Truck'].copy()).astype(int)
Accidents['Big_Truck'] = (Accidents['Big_Truck'].copy()).astype(int)
Accidents['Way'] = (Accidents['Way'].copy()).astype(int)
Accidents['Paints'] = (Accidents['Paints'].copy()).astype(int)
Accidents['Animal'] = (Accidents['Animal'].copy()).astype(int)

Accidents['Accident_Type'] = (Accidents['Accident_Type'].copy()).astype(str)
Accidents['Road_Status'] = (Accidents['Road_Status'].copy()).astype(str)
Accidents['Road_Close_Days'] = (Accidents['Road_Close_Days'].copy()).astype(int)
Accidents['Road_Type'] = (Accidents['Road_Type'].copy()).astype(str)
Accidents['Damage_Road_Type'] = (Accidents['Damage_Road_Type'].copy()).astype(str)
Accidents['GEOMETRIC_RoadTYPE'] = (Accidents['GEOMETRIC_RoadTYPE'].copy()).astype(str)
Accidents['Weather_Status'] = (Accidents['Weather_Status'].copy()).astype(str)
Accidents['Region'] = (Accidents['Region'].copy()).astype(str)

In [141]:
# This segment is for Quality checking of Date/Time

Accident_Date_Time = Accidents.Accident_Date_G + ' ' + Accidents.Accident_Time
Accidents['Accident_Date_G_Time'] = [parse(x) for x in Accident_Date_Time].copy()
Accidents[['Accident_Date_G','Accident_Time','Accident_Date_G_Time']]

,Accident_Date_G,Accident_Time,Accident_Date_G_Time
0,16/04/2019,5:00 AM,2019-04-16 05:00:00
1,16/04/2019,12:15 PM,2019-04-16 12:15:00
2,15/04/2019,8:30 AM,2019-04-15 08:30:00
3,15/04/2019,8:30 PM,2019-04-15 20:30:00
4,14/04/2019,12:10 AM,2019-04-14 00:10:00
5,14/04/2019,5:30 AM,2019-04-14 05:30:00
6,14/04/2019,11:45 AM,2019-04-14 11:45:00
7,14/04/2019,2:30 PM,2019-04-14 14:30:00
8,14/04/2019,4:50 PM,2019-04-14 16:50:00
9,14/04/2019,10:30 PM,2019-04-14 22:30:00


In [142]:
# another round of missing value checks
# verification for missing values for all columns
# Count the missing values — sum() works because True is 1 and False is 0.
Accidents_dic = {}
# show all columns with missing count
for one_col in Accidents.columns:
    # assign value to the key value calculate the total missing values then it to dict
    Accidents_dic[one_col] = "missing " + str(Accidents[one_col].isnull().sum())
print(Accidents_dic)
#Accidents_dic.keys()

{'Region': 'missing 0', 'lat': 'missing 0', 'Road_Status': 'missing 0', 'Other_Car': 'missing 0', 'Car': 'missing 0', 'Animal': 'missing 0', 'Road_Close_Days': 'missing 0', 'SequenceNo': 'missing 0', 'Vehicles_Number': 'missing 0', 'Accident_Date_G_Time': 'missing 0', 'Damage_Road_Type': 'missing 0', 'Small_Truck': 'missing 0', 'Big_Truck': 'missing 0', 'Eyes': 'missing 0', 'Accident_Type': 'missing 0', 'GEOMETRIC_RoadTYPE': 'missing 0', 'Accident_Time': 'missing 0', 'Weather_Status': 'missing 0', 'Paints': 'missing 0', 'Accident_Date_G': 'missing 0', 'Dead_Number': 'missing 0', 'Driver': 'missing 0', 'Road_Close_Hours': 'missing 0', 'Way': 'missing 0', 'Injuries_Number': 'missing 0', 'long': 'missing 0', 'Vehicle': 'missing 0', 'Rno': 'missing 0', 'Road_Type': 'missing 0'}


In [148]:
#save for visualization Tablaue
"""writer = ExcelWriter('Accidents.xlsx')
Accidents.to_excel(writer,'Sheet')
writer.save()
# DF TO CSV
Accidents.to_csv('Accidents.csv', sep=',')"""

In [152]:
#Take a back-up in case
Accidents_bu = Accidents.copy()

In [153]:
Accidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36016 entries, 0 to 36015
Data columns (total 29 columns):
SequenceNo              36016 non-null int64
Accident_Time           36016 non-null object
Accident_Date_G         36016 non-null object
Rno                     36016 non-null int64
Road_Type               36016 non-null object
Region                  36016 non-null object
Road_Status             36016 non-null object
Weather_Status          36016 non-null object
Dead_Number             36016 non-null int32
Injuries_Number         36016 non-null int32
Damage_Road_Type        36016 non-null object
Accident_Type           36016 non-null object
Car                     36016 non-null int32
Small_Truck             36016 non-null int32
Big_Truck               36016 non-null int32
Other_Car               36016 non-null int32
Driver                  36016 non-null int32
Way                     36016 non-null int32
Vehicle                 36016 non-null int32
Animal                  3601